# Funciones SPETo
***

_Autor:_    __Jesús Casado__ <br> _Revisión:_ __12/05/2019__ <br>

__Introducción__<br>
Funciones para trabajar con la base de datos SPETo: https://doi.org/10.5194/essd-11-1917-2019

__Cosas que arreglar__ <br>

***

__Índice__ <br>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import seaborn as sns
from netCDF4 import Dataset
import datetime

In [ ]:
def extract_SPETo(variable, lonlim=None, latlim=None, datelim=None, coordsIn='epsg:25830', coordsOut=None,
                  dateformat='%Y-%m-%d', plot=False, **kwargs):
    """Extrae los datos de Spain02 v5 para la variable y espacio definidos.
    
    Entradas:
    ---------
    variable:    string. Abreviatura de la variable en la base de datos: 'pr', precipitación; 'tas', temperatura...
    lonlim:      list(2,1). Límites de longitud del rectángulo de búsqueda. Si es 'None' se extraen todas las longitudes disponibles
    latlim:      list(2,1). Límites de latitud del rectángulo de búsqueda. Si es 'None' se extraen todas las latitudes disponibles
    datelim:     list of strings (2,1). Fechas límites de la serie a extraer. Si es 'None' se extraen todas las fechas disponibles
    coordsIn:    string. Código EPSG del sistema de coords de 'lonlim' y 'latlim'. Por defecto UTM ETRS89-30N
    coordsOut:   string. Código EPSG del sistema de coords de los datos de salida. Por defecto (None) se toma el mismo que 'coordsIn'
    dateformat:  string. Formato de las fechas en 'datelim'
    plot:        boolena. Si se quiere plotear la serie
    
    Salidas:
    --------
    Como métodos de la función:
    data:        array 3D [dates, lat, lon]. Datos extraídos
    dates:       array 1D. Fechas en 'data'
    Si las coordenadas son geográficas:
    lat:         array 1D. Latitudes en 'data'
    lon:         array 1D. Longitudes en 'data'
    Si las coordenadas son proyectadas:
    X:           array 2D. Coordenada x de cada celda del mapa diario
    Y:           array 2D. Coordenada y de cada celda del mapa diario
    """
    
    # comprobaciones previas: 
    if (lonlim != None) & (latlim != None):
        if (len(lonlim) != 2) | (len(latlim) != 2):
            print('ERROR. Longitud errónea de "latlim" o "lonlim".')
            return
        if coordsIn != 'epsg:25830':
            ETRS89_30N = Proj(init='epsg:25830')
            SistRefIn = Proj(init=coordsIn)
            lonlim, latlim = transform(SistRefIn, ETRS89_30N, lonlim, latlim)

    # cargar NetCDF
    nc = Dataset('G:/CLIMA/02_HYDRO-CLIMATE/Sp_1km_ETo/' + variable + '.nc', 'r')
    data = nc[variable][::].copy()
    data = np.flip(data, axis=1)

    # variable tiempo (fechas)
    time = nc['time'][::].data
    time_ref = datetime.date(1970, 1, 1)
    dates = [time_ref + datetime.timedelta(days=t) for t in time]
    if datelim != None:
        if len(datelim) != 2:
            print('ERROR. Longitud errónea de "datelim".')
            return
        datelim = [datetime.datetime.strptime(lim, dateformat).date() for lim in datelim]
        maskdate = (dates >= datelim[0]) & (dates <= datelim[-1])
        extract_SPETo.dates = dates[maskdate]
        data = data[maskdate, :, :]
    else:
        extract_SPETo.dates = dates

    # variable latitud
    lat = nc['lat'][::-1].data
    if latlim != None:
        masklat = (lat >= latlim[0]) & (lat <= latlim[-1])
        data = data[:, masklat, :]
        lat = lat[masklat]

    # variable longitud
    lon = nc['lon'][::].data
    if lonlim != None:
        masklon = (lon >= lonlim[0]) & (lon <= lonlim[-1])
        data = data[:, :, masklon]
        lon = lon[masklon]

    # guardar 'array' con datos extraídos
    extract_SPETo.data = data

    # guardar latitud y longitud en el sistema 'coords'
    if coordsOut == None:
        coordsOut = coordsIn
    if coordsOut != 'epsg:25830':
        # matrices de longitud y latitud de cada una de las celdas
        lonaux, lataux = np.meshgrid(lon, lat)
        # transformar coordendas y reformar en matrices del mismo tamaño que el mapa diario
        SistRefOut = Proj(init=coordsOut)
        xaux, yaux = transform(ETRS89_30N, SistRefOut, lonaux.flatten(), lataux.flatten())
        extract_SPETo.X = xaux.reshape(lonaux.shape)
        extract_SPETo.Y = yaux.reshape(lataux.shape)
    else:
        extract_SPETo.lat = lat
        extract_SPETo.lon = lon

    if plot == True:
        # extraer kwargs
        figsize = kwargs['figsize'] if 'figsize' in kwargs else (14, 3)
        color = kwargs['color'] if 'color' in kwargs else 'steelblue'
        fontsize = kwargs['fontsize'] if 'fontsize' in kwargs else 12
        
        fig, ax = plt.subplots(figsize=figsize)
        for i in range(data.shape[1]):
            for j in range(data.shape[2]):
                ax.plot(dates, data[:, i, j], lw=.2, c=color, alpha=.1)
        ax.tick_params(labelsize=fontsize - 2)
        ax.set(xlim=(dates[0], dates[-1]))
        ylabel = {'ETo_Ra': 'ETo_Ra (mm)', 'ETo': 'ETo (mm)', 'ETo_Ae': 'ETo_Ae (mm)', 'ETo_var': 'ETo_var (mm²)'}
        ax.set_ylabel(ylabel[variable], fontsize=fontsize)
        ax.set_title('SPETo', fontsize=fontsize + 1, fontweight='bold');